<center><h1> AI Fairness 360 </h1></center>

-----------------------------------

Welcome to the handson!!!

- In this handson you will be **detecting and mitigating gender bias on adult dataset using AI Fairness 360 (aif360)**
- You will be implementing **BinaryLabelDatasetMetric** to detect bias and **DisparateImpactRemover algorithm** to mitigate bias in the dataset
- Follow the instruction provided for cell to write the code in each cell.
- Run the below cell for to import necessary packages to read
- Before submit your notebook. Restart the kernel and run all the cell. Make sure that any cell shouldn't cause any error or problem.
- Don't forget to run the last cell in the jupyter notebook, failing which your efforts will be invalid.
- Don't delete any cell given in the notebook.

In this scenario, will utilize AI Fairness 360 (aif360) to detect and mitigate bias. We will use the AdultDataset, splitting it into a training and test dataset. We will look for bias in the creation of a machine learning model to predict if user has income-per-year > 50k. The protected attribute will be "gender", with "1" (Male) and "0" (Female) being the values for the privileged and unprivileged groups, respectively.

In [122]:
import numpy as np

from aif360.datasets import AdultDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.algorithms.preprocessing import DisparateImpactRemover

from sklearn.linear_model import LogisticRegression

from test_aifairness import bias_mitigation

np.random.seed(0)

**Question 1: Load AdultDataset dataset and set the protected attribute to be gender. Keep the `age`, `education-num`, `capital-gain`, `capital-loss`, `hours-per-week` features. Change protected attribute "gender" will be, with "1" (Male) and "0" (Female) being the values for the privileged and unprivileged groups.**

- Assign dataset into the variable `data`

In [123]:
label_map=[{1: '>50K', 0: '<=50K'}],
protected_attribute_maps= [{1: 'Male', 0: 'Female'}]
data = AdultDataset(protected_attribute_names=['gender'],privileged_classes=[['Male']],categorical_features=[],features_to_keep=[ 'age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week'])#, metadata={'label_map': label_map,'protected_attribute_maps': protected_attribute_maps})
#print(data.convert_to_dataframe()[0].shape)

**Question 2: Split the original dataset into training and testing datasets *`(Use attributes: num_or_size_splits = [0.8], shuffle = True and seed = 35`* and Set two variables `privileged_groups` and `unprivileged_groups` for the privileged (1) and unprivileged (0) values for the gender attribute. (Example: `group = [{attribute: value}]`)**

- Assign train dataset into the variable `data_train`
- Assign test dataset into the variable `data_test`
- Assign privileged_groups into the variable `privileged_groups`
- Assign unprivileged_groups into the variable `unprivileged_groups`

In [124]:
data_train, data_test = data_train, data_test = data.split([0.8], shuffle=True,seed = 35)
'''from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(copy=False)
data_train.features = scaler.fit_transform(data_train.features)
data_test.features = scaler.fit_transform(data_test.features)'''
privileged_groups = [{'gender': 1}]
unprivileged_groups = [{'gender': 0}]

### Compute fairness metric on original training dataset
--------------------------------

Will use the training dataset and testing dataset in this handson, a normal workflow would also use a test dataset for assessing the efficacy (accuracy, fairness, etc.) during the development of a machine learning model.

we've identified the protected attribute 'gender' and defined privileged and unprivileged values, we can use aif360 to detect bias in the dataset.

**Question 3: Compute fairness metric by calculating disparate_impact for privileged and unprivileged groups using original train dataset (Hint: Use BinaryLabelDatasetMetric).**

- Assign disparate impact value into the variable `disparate_impact_val`

In [125]:
metric_train = BinaryLabelDatasetMetric(data_train, unprivileged_groups=unprivileged_groups,privileged_groups=privileged_groups)
disparate_impact_val = metric_train.disparate_impact()

In [126]:
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % disparate_impact_val)

if disparate_impact_val < 0.8:
    print("\n\nThe industry standard is a four-fifths rule: if the unprivileged group receives a positive outcome less than 80% of their proportion of the privilege group, this is a disparate impact violation. However, you may decide to increase this for your business. In this scenario, we are below the threshold of 0.8 so we deem this to be unfair.")

Difference in mean outcomes between unprivileged and privileged groups = 0.361211


The industry standard is a four-fifths rule: if the unprivileged group receives a positive outcome less than 80% of their proportion of the privilege group, this is a disparate impact violation. However, you may decide to increase this for your business. In this scenario, we are below the threshold of 0.8 so we deem this to be unfair.


**Run the below cell, to split `X_train`, `X_test`, `y_train`, `y_test`**

In [127]:
index = data_train.feature_names.index('gender')

X_train = np.delete(data_train.features, index, axis=1)
X_test = np.delete(data_test.features, index, axis=1)
y_train = data_train.labels.ravel()
y_test = data_test.labels.ravel()

**Question 4: Train the machine learning model and predict the income-per-year > 50k. (Hint: Use LogisticRegression with `class_weight='balanced', solver='liblinear'`).**

- Assign predicted value into the variable `predict_val`

In [128]:
lr = LogisticRegression(solver='liblinear',class_weight='balanced')
model = lr.fit(X_train, y_train)
predict_val = model.predict(X_test)

**Let's Compute fairness metric by calculating disparate_impact for privileged and unprivileged groups using predicted values (Hint: Use BinaryLabelDatasetMetric).**

In [129]:
test_pred = data_test.copy()
test_pred.labels = predict_val

metric_predict = BinaryLabelDatasetMetric(test_pred, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
disparate_impact_val2 = metric_predict.disparate_impact()

print("Difference in mean outcomes between unprivileged and privileged groups = %f" % disparate_impact_val2)

if disparate_impact_val2 < 0.8:
    print("\n\nAs with our initial data, we see a substantially greater proportion of the privileged group recieving the favourable output and identify this to be unfair.")

Difference in mean outcomes between unprivileged and privileged groups = 0.649110


As with our initial data, we see a substantially greater proportion of the privileged group recieving the favourable output and identify this to be unfair.


###  Mitigate bias by transforming the original dataset
-------------------------------
The previous step showed that the privileged group was getting more positive outcomes in the predict values. Since this is not desirable, we are going to try to mitigate this bias.

**Question 5: Mitigate bias by transforming the original training and testing dataset (Hint: Use DisparateImpactRemover algorithm).**

- Assign transformed training dataset into the variable `data_transf_train`
- Assign transformed testing dataset into the variable `data_transf_test`

In [130]:
di = DisparateImpactRemover(repair_level=1.0)
data_transf_train = di.fit_transform(data_train)
data_transf_test = di.fit_transform(data_test)

**Run the below cell, to split `X_train`, `X_test`, `y_train`, `y_test`**

In [131]:
index = data_transf_train.feature_names.index('gender')

X_train = np.delete(data_transf_train.features, index, axis=1)
X_test = np.delete(data_transf_test.features, index, axis=1)
y_train = data_transf_train.labels.ravel()
y_test = data_transf_test.labels.ravel()

### Compute fairness metric on transformed dataset
--------------------------------

Now that we have a transformed dataset, we can check how effective it was in removing bias by using the same metric

**Question 6: Now Train the machine learning model using transformed dataset (`X_train`, `X_test`, `y_train`, `y_test`) (Hint: Use LogisticRegression with `class_weight='balanced', solver='liblinear'`) and Compute fairness metric by calculating mean difference for privileged and unprivileged groups using predicted values (Hint: Use BinaryLabelDatasetMetric).**

- Assign disparate impact value into the variable `disparate_impact_val_di`

In [132]:
# Please refer above cells for calculating fairness value for predicted values
lr = LogisticRegression(solver='liblinear', class_weight='balanced')
model = lr.fit(X_train, y_train)
predict_val = model.predict(X_test)
test_pred = data_test.copy()
test_pred.labels = predict_val
metric_predict = BinaryLabelDatasetMetric(test_pred, privileged_groups=privileged_groups, unprivileged_groups=unprivileged_groups)
disparate_impact_val_di = metric_predict.disparate_impact()

In [133]:
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % disparate_impact_val_di)

if disparate_impact_val_di > 0.8:
    print("\n\nNow it is fair!")

Difference in mean outcomes between unprivileged and privileged groups = 0.900934


Now it is fair!


### Run the below cells to save your answers


In [134]:
bias_mitigation.save_ans1(data)
bias_mitigation.save_ans2(data_train, data_test, privileged_groups, unprivileged_groups)
bias_mitigation.save_ans3(disparate_impact_val)
bias_mitigation.save_ans4(predict_val)
bias_mitigation.save_ans5(data_transf_train, data_transf_test)
bias_mitigation.save_ans6(disparate_impact_val_di)